## Main Notebook -> For Running Alignment (Stage-1 Training)

This notebook is used for alignment training of the models

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from imports.configs.config import setup_from_yaml
from imports.dataset import PixmoFeatureDataset, LibriSpeechFeatureDataset, collate_alignment
from imports.perceiver import PerceiverLatentEncoder, ProjectorMLP
from imports.align_training.text_encoder import HFTextEncoderConfig, HFTextEncoder
from imports.align_training.steps import AlignmentModules, AlignmentConfig
from imports.align_training.training import build_alignment_optimizer, train_alignment

In [14]:
cfg = setup_from_yaml("imports/configs/config.yaml")  # uses your Config + YAML loader
device = cfg.torch_device
dtype = cfg.torch_dtype
print("Device:", device, "dtype:", dtype)

[Config] Device: cuda, dtype: torch.float16
[Config] root_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass
[Config] features_dir: /storage/ice1/1/0/vchopra37/projects/edge_glass/code_base/v1_code_base/edge_glass/features
Device: cuda dtype: torch.float16


### 6.2 Build datasets & dataloaders

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(cfg.models.llm_model_name)

# Vision (PixMo) features
pixmo_train = PixmoFeatureDataset(cfg.datasets.pixmo_train_index)
pixmo_val   = PixmoFeatureDataset(cfg.datasets.pixmo_val_index)

# Optional small subsets for quick testing
if cfg.training.train_subset_size and cfg.training.train_subset_size < len(pixmo_train):
    from torch.utils.data import Subset
    pixmo_train = Subset(pixmo_train, range(cfg.training.train_subset_size))

if cfg.training.val_subset_size and cfg.training.val_subset_size < len(pixmo_val):
    from torch.utils.data import Subset
    pixmo_val = Subset(pixmo_val, range(cfg.training.val_subset_size))


[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/train_feat_769.pt
[PixmoFeatureDataset] Loaded 872 valid entries from 873 total.
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_97.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_98.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_8.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_10.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_19.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_21.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_55.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/val_feat_57.pt
[PixmoFeatureDataset] WARNING: skipping missing feature file: 

In [16]:
from imports.dataset import PixmoFeatureDataset

ds_test = PixmoFeatureDataset(cfg.datasets.pixmo_train_index)
len(ds_test)  # should be <= original, but all entries valid now

# sanity check a few random samples
for i in [0, 10, 100]:
    if i >= len(ds_test):
        break
    rec = ds_test.index[i]
    print(i, rec["resolved_path"])


[PixmoFeatureDataset] WARNING: skipping missing feature file: data/pixmo/features/train_feat_769.pt
[PixmoFeatureDataset] Loaded 872 valid entries from 873 total.
0 data/data/pixmo/features/train_feat_64.pt
10 data/data/pixmo/features/train_feat_77.pt
100 data/data/pixmo/features/train_feat_20.pt


In [17]:
# The code snippet you provided is setting up data loaders for training and validation data using the `DataLoader` class. These data loaders are used to load batches of data for training a machine learning model.
# Nvidia

vision_train_loader = DataLoader(
    pixmo_train,
    batch_size=cfg.training.batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=lambda b: collate_alignment(b, tokenizer),
)

vision_val_loader = DataLoader(
    pixmo_val,
    batch_size=cfg.training.batch_size,
    shuffle=False,
    num_workers=4,
    collate_fn=lambda b: collate_alignment(b, tokenizer),
)



In [18]:
# from functools import partial

# collate_fn = partial(collate_alignment, tokenizer=tokenizer)

# vision_train_loader = DataLoader(
#     pixmo_train,
#     batch_size=cfg.training.batch_size,
#     shuffle=True,
#     num_workers=0,  # <-- IMPORTANT for now on macOS
#     collate_fn=collate_fn,
# )

# vision_val_loader = DataLoader(
#     pixmo_val,
#     batch_size=cfg.training.batch_size,
#     shuffle=False,
#     num_workers=0,  # <-- same here
#     collate_fn=collate_fn,
# )




In [19]:
train_loaders = {
    "vision": vision_train_loader,
    # "audio": audio_train_loader,  # add later if you want
}


### 6.3 Build text encoder

In [20]:
txt_cfg = HFTextEncoderConfig(
    model_name=cfg.models.llm_model_name,
    max_length=128,
    trainable=False,  # Stage-1: keep frozen
)

text_encoder = HFTextEncoder(
    cfg=txt_cfg,
    device=device,
    dtype=dtype,
)

d_text = text_encoder.hidden_size
print("Text hidden size:", d_text)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Text hidden size: 2048


In [21]:
def text_embed_fn(texts: list[str], max_length: int) -> torch.Tensor:
    # We could override max_length by rebuilding text_encoder, but usually
    # HFTextEncoderConfig.max_length is enough, so we ignore this arg.
    with torch.no_grad():
        return text_encoder.encode(texts).to(device)


### 6.4 Build adapters, Perceiver, projector

In [22]:
# Peek one example to get feature dim
sample = pixmo_train[0] if not isinstance(pixmo_train, torch.utils.data.Subset) else pixmo_train.dataset[pixmo_train.indices[0]]
d_feat_v = sample["features"].shape[-1]
print("Vision feature dim:", d_feat_v)


Vision feature dim: 1536


In [23]:
d_perceiver = cfg.architecture.perceiver_dim or d_feat_v
print("Perceiver dim:", d_perceiver)


Perceiver dim: 1536


In [24]:
vision_adapter = nn.Linear(d_feat_v, d_perceiver).to(device=device, dtype=dtype)

perceiver = PerceiverLatentEncoder(
    num_latents=cfg.architecture.num_latents,
    d_latent=d_perceiver,
    d_input=d_perceiver,
    num_layers=cfg.architecture.num_perceiver_layers,
    num_heads=cfg.architecture.num_attn_heads,
    mlp_ratio=cfg.architecture.mlp_ratio,
    dropout=0.1,
).to(device=device, dtype=dtype)

projector = ProjectorMLP(
    d_in=d_perceiver,
    d_out=d_text,
    hidden_factor=2.0,
    dropout=0.1,
).to(device=device, dtype=dtype)

modules = AlignmentModules(
    vision_adapter=vision_adapter,
    audio_adapter=None,  # add audio later if needed
    perceiver=perceiver,
    projector=projector,
)


### 6.5 Alignment config + optimizer

In [25]:
# MRL radii from config; default to full dim if None
mrl_dims = tuple(cfg.mrl.mrl_dims) if cfg.mrl.mrl_dims is not None else (d_text,)

align_cfg = AlignmentConfig(
    mrl_dims=mrl_dims,
    mrl_temperature=cfg.mrl.mrl_temp,
    max_text_length=64,  # arbitrary; text_encoder already truncates internally
)

optimizer = build_alignment_optimizer(
    modules=modules,
    learning_rate=float(cfg.training.learning_rate),
    weight_decay=float(cfg.training.weight_decay),
)
print("Trainable params:", sum(p.numel() for p in optimizer.param_groups[0]["params"]))


Trainable params: 174032384


### 6.6 Run training

In [26]:
mean_epochs = cfg.training.num_epochs

train_alignment(
    train_loaders=train_loaders,
    modules=modules,
    cfg=align_cfg,
    text_embed_fn=text_embed_fn,
    optimizer=optimizer,
    device=device,
    num_epochs=mean_epochs,
    log_every=cfg.training.log_every_steps,
    log_fn=None,   # or pass wandb.log
    modalities=("vision",),  # add "audio" when you wire audio
)


[Epoch 0 | vision] step 0050 | loss nan
[Epoch 0] modality=vision | mean_loss=nan
[Epoch 1 | vision] step 0050 | loss nan
[Epoch 1] modality=vision | mean_loss=nan
[Epoch 2 | vision] step 0050 | loss nan
[Epoch 2] modality=vision | mean_loss=nan
[Epoch 3 | vision] step 0050 | loss nan
[Epoch 3] modality=vision | mean_loss=nan
[Epoch 4 | vision] step 0050 | loss nan
[Epoch 4] modality=vision | mean_loss=nan


In [27]:
# mean_epochs = cfg.training.num_epochs

# train_alignment(
#     train_loaders=train_loaders,
#     modules=modules,
#     cfg=align_cfg,
#     text_embed_fn=text_embed_fn,
#     optimizer=optimizer,
#     device=device,
#     num_epochs=mean_epochs,
#     log_every=cfg.training.log_every_steps,
#     log_fn=None,
#     modalities=("vision",),
# )
